# Subset of Network Analysis for ABCD data

http://dx.plos.org/10.1371/journal.pbio.1002328  
https://www.sciencedirect.com/science/article/pii/S105381191730109X?via%3Dihub

In [1]:
# ! pip install python-louvain

In [2]:
import glob
import os
import networkx as nx
import numpy as np
import pandas as pd
import community
from sklearn.metrics.cluster import normalized_mutual_info_score
import bz2
import pickle
import pdb
import statistics
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

from visbrain.objects import ConnectObj, SceneObj, SourceObj, BrainObj
from visbrain.io import download_file

/Users/gracer/.local/lib/python3.6/site-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


## Data of interest (from R notebook)

### Read in labels

In [3]:
labels = pd.read_csv('/Users/gracer/Google Drive/ABCD/important_txt/locations.csv', sep=",")

In [4]:
import pickle
with open('/Users/gracer/Google Drive/ABCD/tmp/graphAna4', 'rb') as pickle_file:
    try:
        while True:
            GRAPHS = pickle.load(pickle_file)
            print (GRAPHS)
    except EOFError:
        pass

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
GRAPHS.keys()

dict_keys(['edges', 'correlations', 'mean_FC', 'graphs', 'BIGdf'])

### analysis picked back up, needed to unpickle

In [6]:
BIGdf=GRAPHS['BIGdf']

### Getting the standard deviation 

In [7]:
GRAPHS['mean_FC']['sub-NDARINVULZMADF1']

[0.4557258893193297,
 0.41891037471303944,
 0.49077049480601465,
 0.48699714124196514,
 'F',
 'latepubertal',
 'Obese',
 0.46310097502008724,
 0.03338577606246018]

### Making dataframe to use 

In [8]:
PID = list(GRAPHS['mean_FC'].keys())
len(PID)

120

In [9]:
sex=[]
PCS=[]
OVOB=[]
mean_FC=[]
sd_FC=[]
for item in list(GRAPHS['mean_FC'].values()):
    sex.append(item[-5])
    PCS.append(item[-4])
    OVOB.append(item[-3])
    mean_FC.append(item[-2])
    sd_FC.append(item[-1])

In [10]:
len(OVOB)

120

### Creating a dataframe 

In [11]:
FCmat=np.column_stack([PID, sex, PCS, OVOB, mean_FC, sd_FC])
FCdf=pd.DataFrame(FCmat, columns = ['subjects','sex','PCS','OVOB','mean','SD'])
FCdf=FCdf.fillna(0)

In [12]:
FCdf.describe()

,subjects,sex,PCS,OVOB,mean,SD
count,120,120,120,120,120,120
unique,120,2,3,3,120,118
top,sub-NDARINV5FBV2BUF,M,latepubertal,Normalweight,0.28372023993485995,NaN
freq,1,63,44,44,1,3


### Testing differences in mean FC
See the matching R notebook
Summary:
* No difference in the FC between PCS (no need to control for it in models)
* Trending difference in the FC between OVOB (may need to control for it in the model)

### Differences in modularity by subject (probs won't use, don't know what it means)

In [13]:
GRAPHS['graphs']['sub-NDARINVULZMADF1'][10]

'Obese'

In [14]:
categories = BIGdf[['subjects', 'sex','PCS','OVOB']]
categories.head()

,subjects,sex,PCS,OVOB
0,sub-NDARINVULZMADF1,F,latepubertal,Obese
1,sub-NDARINVULZMADF1,F,latepubertal,Obese
2,sub-NDARINVULZMADF1,F,latepubertal,Obese
3,sub-NDARINVULZMADF1,F,latepubertal,Obese
4,sub-NDARINVD9L81NY5,F,latepubertal,Obese


In [15]:
x=BIGdf[1].value_counts().idxmax()

In [16]:
maxes={}
for sub in PID:
    subset=BIGdf.loc[(BIGdf['subjects'] == sub)]
    for col in subset:
        macks = subset[col].value_counts().idxmax()
        maxes.setdefault(sub, []).append(macks)
    #     print(PARTdf[col].value_counts().idxmax())

In [17]:
subjects=[]
max_vals=[]
for key, value in maxes.items():
#     print(item.keys())
    subjects.append(key)
    max_vals.append(list(value))

In [18]:
Subar = np.array(subjects)
Valar=np.array(max_vals)
print(Subar.shape)
print(Valar.shape)

(120,)
(120, 270)


In [19]:
MAXdf = pd.DataFrame(Valar)
MAXdf['subjects'] = Subar

In [20]:
# MAXdf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/MAX.csv", sep=',', index=False)

In [21]:
norm_max=[]
subset=MAXdf.loc[(MAXdf[267] == 'Normalweight')]
for col in subset:
    mechs = subset[col].value_counts().idxmax()
    norm_max.append(mechs)
#     print(PARTdf[col].value_counts().idxmax())

In [22]:
ov_max=[]
subset=MAXdf.loc[(MAXdf[267] == 'Overweight')]
for col in subset:
    mechs = subset[col].value_counts().idxmax()
    ov_max.append(mechs)
#     print(PARTdf[col].value_counts().idxmax())

In [23]:
ob_max=[]
subset=MAXdf.loc[(MAXdf[267] == 'Obese')]
for col in subset:
    mechs = subset[col].value_counts().idxmax()
    ob_max.append(mechs)

In [24]:
normAR = np.array(norm_max)
ovAR = np.array(ov_max)
obAR = np.array(ob_max)

print(normAR.shape)
print(ovAR.shape)
print(obAR.shape)

(271,)
(271,)
(271,)


In [25]:
len(ob_max)

271

### Normalized information score  
sklearn.metrics.normalized_mutual_info_score(labels_true [group 1], labels_pred [group 2], average_method=’warn’)

In [26]:
diff_no_ov=normalized_mutual_info_score(norm_max, ov_max)

In [27]:
diff_no_ob=normalized_mutual_info_score(norm_max, ob_max)

In [28]:
diff_ov_ob=normalized_mutual_info_score(ov_max, ob_max)

In [29]:
diff_no_ob

0.4244090131216969

In [30]:
diff_no_ov

0.4249421843186356

In [31]:
diff_ov_ob

0.3698487741255595

##  Create a Mega graph by OVOB and PCS

In [32]:
def make_total_graphs(dict_o_data):
    mylist=[]
    for key, val_list in dict_o_data.items():
#         print(key)
        for i in val_list:
            cor_matrix = np.asarray(i)
            mylist.append(cor_matrix)
    x=np.stack(mylist, axis=2)
    mu=np.mean(x, axis=(2))
    return(mu)

In [33]:
GRAPHS.keys()

dict_keys(['edges', 'correlations', 'mean_FC', 'graphs', 'BIGdf'])

In [34]:
list(GRAPHS['correlations'].values())[1]

[matrix([[0.      , 0.297177, 0.366261, ..., 0.712325, 0.027972, 0.032292],
         [0.297177, 0.      , 1.067256, ..., 0.267867, 0.482737, 0.50939 ],
         [0.366261, 1.067256, 0.      , ..., 0.583668, 0.510977, 0.520726],
         ...,
         [0.712325, 0.267867, 0.583668, ..., 0.      , 0.253103, 0.211843],
         [0.027972, 0.482737, 0.510977, ..., 0.253103, 0.      , 1.947065],
         [0.032292, 0.50939 , 0.520726, ..., 0.211843, 1.947065, 0.      ]]),
 matrix([[ 0.      ,  0.618682,  0.288411, ...,  0.212559,  0.336378,
           0.047735],
         [ 0.618682,  0.      ,  0.972095, ...,  0.283186,  0.510952,
           0.318188],
         [ 0.288411,  0.972095, 18.714974, ...,  0.419613,  0.701422,
           0.603747],
         ...,
         [ 0.212559,  0.283186,  0.419613, ...,  0.      ,  0.383667,
           0.340115],
         [ 0.336378,  0.510952,  0.701422, ...,  0.383667,  0.      ,
           1.268736],
         [ 0.047735,  0.318188,  0.603747, ...,  0.340

In [35]:
no_subset=categories.loc[(categories['OVOB'] == 'Normalweight')]
no_dict = {k: GRAPHS['correlations'][k] for k in no_subset['subjects'] if k in GRAPHS['correlations']}

ov_subset=categories.loc[(categories['OVOB'] == 'Overweight')]
ov_dict = {k: GRAPHS['correlations'][k] for k in ov_subset['subjects'] if k in GRAPHS['correlations']}

ob_subset=categories.loc[(categories['OVOB'] == 'Obese')]
ob_dict = {k: GRAPHS['correlations'][k] for k in ob_subset['subjects'] if k in GRAPHS['correlations']}


In [36]:
early_subset=categories.loc[(categories['PCS'] == 'earlypubertal')]
early_dict = {k: GRAPHS['correlations'][k] for k in early_subset['subjects'] if k in GRAPHS['correlations']}

late_subset=categories.loc[(categories['PCS'] == 'latepubertal')]
late_dict = {k: GRAPHS['correlations'][k] for k in late_subset['subjects'] if k in GRAPHS['correlations']}

mid_subset=categories.loc[(categories['PCS'] == 'midpubertal')]
mid_dict = {k: GRAPHS['correlations'][k] for k in mid_subset['subjects'] if k in GRAPHS['correlations']}


In [37]:
mean_no=make_total_graphs(no_dict)
mean_ov=make_total_graphs(ov_dict)
mean_ob=make_total_graphs(ob_dict)

In [38]:
mean_early=make_total_graphs(early_dict)
mean_late=make_total_graphs(late_dict)
mean_mid=make_total_graphs(mid_dict)

#### Creates graph using the data of the correlation matrix

In [39]:
noG = nx.from_numpy_matrix(mean_no)
ovG = nx.from_numpy_matrix(mean_ov)
obG = nx.from_numpy_matrix(mean_ob)
OVOBgraphs = [noG, ovG, obG]

In [40]:
earlyG = nx.from_numpy_matrix(mean_early)
midG = nx.from_numpy_matrix(mean_mid)
lateG = nx.from_numpy_matrix(mean_late)
PCSgraphs = [earlyG, midG, lateG]

In [41]:
for graph in OVOBgraphs:
    for i, nlrow in labels.iterrows():
        graph.node[i].update(nlrow[0:].to_dict())

In [42]:
for graph in PCSgraphs:
    for i, nlrow in labels.iterrows():
        graph.node[i].update(nlrow[0:].to_dict())

In [43]:
no_edges,no_weights = zip(*nx.get_edge_attributes(noG,'weight').items())
ov_edges,ov_weights = zip(*nx.get_edge_attributes(ovG,'weight').items())
ob_edges,ob_weights = zip(*nx.get_edge_attributes(obG,'weight').items())

In [44]:
early_edges, early_weights = zip(*nx.get_edge_attributes(earlyG,'weight').items())
mid_edges,mid_weights = zip(*nx.get_edge_attributes(midG,'weight').items())
late_edges,late_weights = zip(*nx.get_edge_attributes(lateG,'weight').items())

In [45]:
for graph in OVOBgraphs:
    d = nx.degree(graph)
    list(d)[0]
    nodelist, node_sizes = zip(*list(d))

In [46]:
for graph in PCSgraphs:
    d = nx.degree(graph)
    list(d)[0]
    nodelist, node_sizes = zip(*list(d))

In [47]:
def create_corr_network(G, corr_direction, min_correlation):
    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
                
    
    #crates a list for edges and for the weights
    edges,weights = zip(*nx.get_edge_attributes(H,'weight').items())
    
    ### increases the value of weights, so that they are more visible in the graph
    weights = tuple([(1+abs(x))**1 for x in weights])
    
    #####calculates the degree of each node
    d = nx.degree(H)
    #####creates list of nodes and a list their degrees that will be used later for their sizes
    nodelist, node_sizes = zip(*list(d))
    return(H)

In [48]:
def create_corr_network_1(G):
    #crates a list for edges and for the weights
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())

    #positions
    positions=nx.circular_layout(G)
    
    #Figure size
    plt.figure(figsize=(15,15))

    #draws nodes
    nx.draw_networkx_nodes(G,positions,node_color='#DA70D6',
                           node_size=500,alpha=0.8)
    
    #Styling for labels
    nx.draw_networkx_labels(G, positions, font_size=8, 
                            font_family='sans-serif')
        
    #draws the edges
    nx.draw_networkx_edges(G, positions, edge_list=edges,style='solid')
    
    # displays the graph without axis
    plt.axis('off')
    #saves image
    plt.savefig("part1.png", format="PNG")
    plt.show() 

#create_corr_network_1(G)

In [49]:
noH=create_corr_network(noG, "positive",0)
ovH=create_corr_network(ovG, "positive", 0)
obH=create_corr_network(obG, "positive", 0)
saveme={}
saveme['noH']=noH
saveme['ovH']=ovH
saveme['obH']=obH

In [50]:
earlyH=create_corr_network(earlyG, "positive",0)
midH=create_corr_network(midG, "positive", 0)
lateH=create_corr_network(lateG, "positive", 0)
saveme={}
saveme['earlyH']=earlyH
saveme['midH']=midH
saveme['lateH']=lateH

### Getting partition values with the large graph

In [94]:
noPART = community.best_partition(noH)
ovPART = community.best_partition(ovH)
obPART = community.best_partition(obH)

# noPART.values()

In [99]:
earlyPART = community.best_partition(earlyH)
midPART = community.best_partition(midH)
latePART = community.best_partition(lateH)

# noPART.values()

In [120]:
noPARTvalues = np.asarray(list(noPART.values()))
obPARTvalues = np.asarray(list(obPART.values()))
ovPARTvalues = np.asarray(list(ovPART.values()))

In [121]:
earlyPARTvalues = np.asarray(list(earlyPART.values()))
latePARTvalues = np.asarray(list(latePART.values()))
midPARTvalues = np.asarray(list(midPART.values()))

### Checking the normalized mutual disribution information

In [53]:
diff_no_ov=normalized_mutual_info_score(list(noPART.values()), list(ovPART.values()))
print(diff_no_ov)

0.6622799993391564


In [54]:
diff_no_ob=normalized_mutual_info_score(list(noPART.values()), list(obPART.values()))
print(diff_no_ob)

0.6443749434984919


In [55]:
diff_early_mid=normalized_mutual_info_score(list(earlyPART.values()), list(midPART.values()))
print(diff_early_mid)

0.7467054519979314


In [56]:
diff_early_late=normalized_mutual_info_score(list(earlyPART.values()), list(latePART.values()))
print(diff_early_late)

0.6956449221096923


### Summary so far
Larger difference between the normal weight and obese compared to the normal weight and the overweight

In [57]:
color_dic = {0:'blue',1:'red',2:'green',3:'purple',4:'yellow',5:'pink',6:'black',7:'orange', 8:'cyan'}

In [58]:
PARTS=[noPART, ovPART ,obPART]
for item in PARTS:
    for key, value in item.items():
        item[key]=(color_dic[value])
#     print(item)

In [59]:
for key, value in noPART.items():
#     print("this is the key %s"%key)
#     print(value)
#     print(noH.node(data=True)[key])
    noH.node(data=True)[key].update({'color':value})


In [60]:
#drawing
size = float(len(set(noPART.values())))
print(size)
pos = nx.spring_layout(noH)
count = 0
for com in set(noPART.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in noPART.keys()
                                if noPART[nodes] == com]
    nx.draw_networkx_nodes(noH, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(noH, pos, alpha=0.1)
plt.show()

6.0


In [61]:
for key, value in ovPART.items():
    ovH.node(data=True)[key].update({'color':value})

In [62]:
for key, value in obPART.items():
    obH.node(data=True)[key].update({'color':value})

In [63]:
# list(H.edges(data=True))[0]
# H.edges(data=True)
no_edge_weights = [e[2]['weight'] for e in noH.edges(data=True)]
ov_edge_weights = [e[2]['weight'] for e in ovH.edges(data=True)]
ob_edge_weights = [e[2]['weight'] for e in obH.edges(data=True)]


In [64]:
# Define node positions data structure (dict) for plotting
no_node_colors = {node[0]: (node[1]['color']) for node in noH.nodes(data=True)}
ov_node_colors = {node[0]: (node[1]['color']) for node in ovH.nodes(data=True)}
ob_node_colors = {node[0]: (node[1]['color']) for node in obH.nodes(data=True)}

# Preview of node_positions with a bit of hack (there is no head/slice method for dictionaries).
#dict(list(node_colors.items())[0:5])
#print(G.node(data=True))
no_list_colors=list(no_node_colors.values())
ov_list_colors=list(ov_node_colors.values())
ob_list_colors=list(ob_node_colors.values())

In [85]:
no_list_colors

['blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'red',
 'red',
 'red',
 'red',
 'red',
 'blue',
 'blue',
 'blue',
 'blue',
 'blue',
 'green',
 'blue',
 'blue',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'red',
 'purple',
 'green',
 'green',
 'purple',
 'green',
 'purple',
 'purple',
 'yellow',
 'yellow',
 'yellow',
 'purple',
 'pink',
 'purple',
 'purple',
 'purple',
 'purple',
 'purple',
 'purple',
 'blue',
 'purple',
 'pink',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'yellow',
 'yellow',
 'yellow',
 'yellow

In [65]:
no_node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in noH.nodes(data=True)}
ov_node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in ovH.nodes(data=True)}
ob_node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in obH.nodes(data=True)}

In [66]:
# Define node positions data structure (dict) for plotting
# node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in H.nodes(data=True)}
# node_colors = {node[0]: (node[1]['color']) for node in H.nodes(data=True)}

# Preview of node_positions with a bit of hack (there is no head/slice method for dictionaries).
#dict(list(node_colors.items())[0:5])
#print(G.node(data=True))
# list_colors=list(node_colors.values())

##### PCS

In [67]:
PARTS=[earlyPART, midPART ,latePART]
for item in PARTS:
    for key, value in item.items():
        item[key]=(color_dic[value])
#     print(item)

In [68]:
for key, value in earlyPART.items():
    earlyH.node(data=True)[key].update({'color':value})


In [69]:
earlyPART.items()

dict_items([(0, 'blue'), (1, 'blue'), (2, 'blue'), (3, 'blue'), (4, 'blue'), (5, 'blue'), (6, 'blue'), (7, 'blue'), (8, 'blue'), (9, 'blue'), (10, 'blue'), (11, 'blue'), (12, 'blue'), (13, 'blue'), (14, 'blue'), (15, 'blue'), (16, 'blue'), (17, 'blue'), (18, 'blue'), (19, 'blue'), (20, 'blue'), (21, 'blue'), (22, 'blue'), (23, 'blue'), (24, 'blue'), (25, 'blue'), (26, 'blue'), (27, 'blue'), (28, 'blue'), (29, 'blue'), (30, 'red'), (31, 'green'), (32, 'green'), (33, 'red'), (34, 'green'), (35, 'blue'), (36, 'green'), (37, 'blue'), (38, 'blue'), (39, 'blue'), (40, 'purple'), (41, 'blue'), (42, 'blue'), (43, 'purple'), (44, 'purple'), (45, 'purple'), (46, 'purple'), (47, 'purple'), (48, 'purple'), (49, 'green'), (50, 'green'), (51, 'green'), (52, 'red'), (53, 'red'), (54, 'red'), (55, 'green'), (56, 'red'), (57, 'red'), (58, 'red'), (59, 'green'), (60, 'green'), (61, 'red'), (62, 'yellow'), (63, 'purple'), (64, 'purple'), (65, 'yellow'), (66, 'purple'), (67, 'yellow'), (68, 'yellow'), (69

In [70]:
earlyH.nodes(data=True)

NodeDataView({0: {'X': -7, 'Y': -52, 'Z': 61, 'ID': 'Precuneus', 'color': 'blue'}, 1: {'X': -14, 'Y': -18, 'Z': 40, 'ID': 'Cingulate Gyrus', 'color': 'blue'}, 2: {'X': 0, 'Y': -15, 'Z': 47, 'ID': 'Paracentral Lobule', 'color': 'blue'}, 3: {'X': 10, 'Y': -2, 'Z': 45, 'ID': 'Cingulate Gyrus', 'color': 'blue'}, 4: {'X': -7, 'Y': -21, 'Z': 65, 'ID': 'Medial Frontal Gyrus', 'color': 'blue'}, 5: {'X': -7, 'Y': -33, 'Z': 72, 'ID': 'Postcentral Gyrus', 'color': 'blue'}, 6: {'X': 13, 'Y': -33, 'Z': 75, 'ID': 'Postcentral Gyrus', 'color': 'blue'}, 7: {'X': -54, 'Y': -23, 'Z': 43, 'ID': 'Postcentral Gyrus', 'color': 'blue'}, 8: {'X': 29, 'Y': -17, 'Z': 71, 'ID': 'Precentral Gyrus', 'color': 'blue'}, 9: {'X': 10, 'Y': -46, 'Z': 73, 'ID': 'Postcentral Gyrus', 'color': 'blue'}, 10: {'X': -23, 'Y': -30, 'Z': 72, 'ID': 'Postcentral Gyrus', 'color': 'blue'}, 11: {'X': -40, 'Y': -19, 'Z': 54, 'ID': 'Postcentral Gyrus', 'color': 'blue'}, 12: {'X': 29, 'Y': -39, 'Z': 59, 'ID': 'Postcentral Gyrus', 'color'

In [71]:
blue=[]
for node in noH.nodes(data=True):
    if node[1]['color']=='blue':
        print(node[0])
        blue.append(node[0])
#         pdb.set_trace()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
35
36
37
38
39
41
42
80
123
156
159
160
168
169
170
171
173
177
181
183
212
221
223
225
226
228
230
231


In [72]:
blue_ob=[]
for node in obH.nodes(data=True):
    if node[1]['color']=='blue':
        print(node[0])
        blue_ob.append(node[0])
#         pdb.set_trace()

0
62
67
68
72
73
74
75
76
77
78
79
81
82
116
120
121
122
123
126
129
137
138
139
141
144
145
148
149
159
169
170
172
173
177
182
221
223
225
226
227
230


### Visualize differences

In [73]:
 G = nx.karate_club_graph()
res = [0,1,2,3,4,5, 'parrot'] #I've added 'parrot', a node that's not in G
                              #just to demonstrate that G.subgraph is okay
                              #with nodes not in G.    
pos = nx.spring_layout(G)  #setting the positions with respect to G, not k.
k = G.subgraph(res)  

plt.figure()
nx.draw_networkx(k, pos=pos)

othersubgraph = G.subgraph(range(6,G.order()))
nx.draw_networkx(othersubgraph, pos=pos, node_color = 'b')
plt.show()

In [74]:
# G = nx.karate_club_graph()
# res = [0,1,2,3,4,5, 'parrot'] #I've added 'parrot', a node that's not in G
                              #just to demonstrate that G.subgraph is okay
                              #with nodes not in G.    
pos = nx.spring_layout(noG)  #setting the positions with respect to G, not k.
subNo = noG.subgraph(blue)  

plt.figure()
nx.draw_networkx(subNo, pos=pos, node_color='b')

# othersubgraph = noG.subgraph(range(50,noG.order()))
# nx.draw_networkx(othersubgraph, pos=pos, node_color = 'g')
plt.show()

In [75]:
# G = nx.karate_club_graph()
# res = [0,1,2,3,4,5, 'parrot'] #I've added 'parrot', a node that's not in G
                              #just to demonstrate that G.subgraph is okay
                              #with nodes not in G.    
pos = nx.spring_layout(obG)  #setting the positions with respect to G, not k.
subOB = obG.subgraph(blue_ob)  

plt.figure()
nx.draw_networkx(subOB, pos=pos, node_color='b')

# othersubgraph = noG.subgraph(range(50,noG.order()))
# nx.draw_networkx(othersubgraph, pos=pos, node_color = 'g')
plt.show()

In [76]:
len(list(noG.edges(data=True)))

34980

In [84]:
NOmat=nx.to_numpy_array(G=noG)

In [102]:
OBmat=nx.to_numpy_array(G=obG)

In [118]:
OVmat=nx.to_numpy_array(G=ovG)

In [103]:
EARLYmat=nx.to_numpy_array(G=earlyG)

In [119]:
MIDmat=nx.to_numpy_array(G=midG)

In [104]:
LATEmat=nx.to_numpy_array(G=lateG)

In [ ]:
# !pip install bctpy --user

In [80]:
import os.path as op
import seaborn as sns
import bct

## Participation Coefficient
Partition networks into the modules, calculate the PC per node within each group. Higher PC indicates more distributed between network connectivity, while a PC of 0 signifies a node’s links are completely within its home network (within network).

In [97]:
PCno = bct.participation_coef(W=NOmat, ci= noPARTvalues)

In [105]:
PCob = bct.participation_coef(W=OBmat, ci= obPARTvalues)

In [122]:
PCov = bct.participation_coef(W=OVmat, ci= ovPARTvalues)

In [106]:
PCearly = bct.participation_coef(W=EARLYmat, ci= earlyPARTvalues)

In [123]:
PCmid = bct.participation_coef(W=MIDmat, ci= midPARTvalues)

In [107]:
PClate = bct.participation_coef(W=LATEmat, ci= latePARTvalues)

In [127]:
alls = np.vstack([PCno, PCov, PCob, PCearly, PCmid, PClate])
alls.shape

(6, 264)

In [116]:
PCearly

array([0.76270429, 0.76737731, 0.76723542, 0.76524188, 0.76196659,
       0.76314555, 0.7621215 , 0.76851828, 0.76375518, 0.76231515,
       0.76332254, 0.76525021, 0.76644386, 0.76874294, 0.76241413,
       0.76511135, 0.76374383, 0.76432397, 0.76054857, 0.76381392,
       0.76723839, 0.76423998, 0.76162449, 0.7667454 , 0.7641471 ,
       0.76335211, 0.764657  , 0.76457985, 0.76822066, 0.76744892,
       0.7725968 , 0.77246912, 0.77357729, 0.77196186, 0.77656864,
       0.76215504, 0.7771982 , 0.76066264, 0.76198374, 0.76425846,
       0.7579467 , 0.76015057, 0.76038975, 0.7590807 , 0.75555895,
       0.75472024, 0.75813043, 0.75366274, 0.75512221, 0.77590299,
       0.77827314, 0.7781857 , 0.77259093, 0.77431408, 0.77343065,
       0.77779697, 0.77356028, 0.77438786, 0.77190714, 0.77444656,
       0.77743308, 0.77210076, 0.75644431, 0.74552899, 0.74269198,
       0.76221907, 0.74500291, 0.7624262 , 0.75744456, 0.75747251,
       0.75605494, 0.76891988, 0.76167424, 0.75971282, 0.76273

In [117]:
PClate

array([0.77341753, 0.766566  , 0.76668862, 0.76348636, 0.76116683,
       0.765666  , 0.76500203, 0.7686324 , 0.75937693, 0.76726171,
       0.76373252, 0.76502419, 0.76944165, 0.7674425 , 0.76472894,
       0.7652829 , 0.7630182 , 0.77054398, 0.7603025 , 0.76588445,
       0.7689418 , 0.76392757, 0.76029715, 0.76419017, 0.76181023,
       0.76923782, 0.76578978, 0.76360845, 0.76729394, 0.76601196,
       0.7681182 , 0.76052118, 0.76503145, 0.77594217, 0.77238819,
       0.76137095, 0.76977545, 0.75990731, 0.75867244, 0.76173399,
       0.75156126, 0.75919222, 0.75812947, 0.75546615, 0.75168074,
       0.74891093, 0.75039294, 0.75061769, 0.746664  , 0.77596407,
       0.78009544, 0.77432626, 0.7831506 , 0.78160802, 0.77782606,
       0.77473749, 0.78245423, 0.77937996, 0.7717853 , 0.77069029,
       0.76897163, 0.78005565, 0.78128724, 0.74355901, 0.7338254 ,
       0.77095238, 0.73819543, 0.78117707, 0.78143215, 0.75015743,
       0.74710237, 0.77746137, 0.78276658, 0.77894254, 0.78292

In [132]:
Alls = pd.DataFrame(alls)
Alls

,0,1,2,3,4,5,6,7,8,9,...,254,255,256,257,258,259,260,261,262,263
0,0.793451,0.789641,0.789688,0.787512,0.784590,0.788517,0.787385,0.794856,0.786566,0.788587,...,0.774287,0.782966,0.784498,0.786193,0.796698,0.795246,0.798849,0.798364,0.791596,0.792101
1,0.738108,0.738126,0.737173,0.733827,0.732678,0.735483,0.734223,0.740619,0.733345,0.736016,...,0.712491,0.728448,0.726784,0.728059,0.724762,0.725015,0.721231,0.726549,0.735714,0.733143
2,0.745952,0.745830,0.744383,0.742126,0.741389,0.742609,0.741479,0.748687,0.740112,0.742648,...,0.716834,0.739876,0.738750,0.741286,0.732191,0.732095,0.730920,0.733884,0.748014,0.747030
3,0.762704,0.767377,0.767235,0.765242,0.761967,0.763146,0.762121,0.768518,0.763755,0.762315,...,0.745182,0.749381,0.750723,0.753880,0.767289,0.766139,0.767301,0.766363,0.760874,0.762056
4,0.802934,0.810630,0.808457,0.806950,0.802036,0.799193,0.797866,0.810891,0.801408,0.799559,...,0.814540,0.801694,0.807087,0.805591,0.821353,0.825038,0.825347,0.825679,0.812349,0.809710
5,0.773418,0.766566,0.766689,0.763486,0.761167,0.765666,0.765002,0.768632,0.759377,0.767262,...,0.739903,0.762935,0.761658,0.763878,0.753239,0.752058,0.749529,0.755270,0.766879,0.768590


In [131]:
Alls.to_csv("/Users/gracer/Google Drive/ABCD/tmp/PC.csv", sep=',', index=False)

In [ ]:
# list(H.edges(data=True))[0]
# H.edges(data=True)
early_edge_weights = [e[2]['weight'] for e in earlyH.edges(data=True)]
mid_edge_weights = [e[2]['weight'] for e in midH.edges(data=True)]
late_edge_weights = [e[2]['weight'] for e in lateH.edges(data=True)]


In [ ]:
early_edge_keep = []
for e in earlyH.edges(data=True):
    if e[2]['weight']>=1:
        early_edge_keep.append(e)

In [ ]:
early_edge_keep

In [ ]:
#drawing
size = float(len(set(earlyPART.values())))
print(size)
pos = nx.spring_layout(earlyH)
count = 0.
for com in set(earlyPART.values()) :
    print(com)
    count = count + 1.
    list_nodes = [nodes for nodes in earlyPART.keys()
                                if earlyPART[nodes] == com]
    nx.draw_networkx_nodes(earlyH, pos, list_nodes, node_size = 20,edge_vmin = 0.5, 
                                node_color = no_list_colors)


nx.draw_networkx_edges(earlyH, pos, alpha=0.5, edgelist=early_edge_keep)
plt.show()

In [ ]:
for key, value in midPART.items():
    midH.node(data=True)[key].update({'color':value})

In [ ]:
midH.nodes(data=True)

In [ ]:
for key, value in latePART.items():
    lateH.node(data=True)[key].update({'color':value})

In [ ]:
# Define node positions data structure (dict) for plotting
early_node_colors = {node[0]: (node[1]['color']) for node in earlyH.nodes(data=True)}
mid_node_colors = {node[0]: (node[1]['color']) for node in midH.nodes(data=True)}
late_node_colors = {node[0]: (node[1]['color']) for node in lateH.nodes(data=True)}

# Preview of node_positions with a bit of hack (there is no head/slice method for dictionaries).
#dict(list(node_colors.items())[0:5])
#print(G.node(data=True))
early_list_colors=list(early_node_colors.values())
mid_list_colors=list(mid_node_colors.values())
late_list_colors=list(late_node_colors.values())

In [ ]:
early_node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in earlyH.nodes(data=True)}
mid_node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in midH.nodes(data=True)}
late_node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in lateH.nodes(data=True)}

In [ ]:
# # Define node positions data structure (dict) for plotting
# node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in H.nodes(data=True)}
# node_colors = {node[0]: (node[1]['color']) for node in H.nodes(data=True)}

# # Preview of node_positions with a bit of hack (there is no head/slice method for dictionaries).
# #dict(list(node_colors.items())[0:5])
# #print(G.node(data=True))
# list_colors=list(node_colors.values())

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(noH, pos=no_node_positions,  node_size=20,  edge_color = "lightgrey", node_color = no_list_colors)
plt.title('normal', size=15)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(earlyH, pos=early_node_positions,  node_size=20,  edge_color = "lightgrey", node_color = early_list_colors)
plt.title('early', size=15)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(obH, pos=ob_node_positions,  node_size=20,  edge_color = "lightgrey", node_color = ob_list_colors)
plt.title('obese', size=15)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(midH, pos=mid_node_positions,  node_size=20,  edge_color = "lightgrey", node_color = mid_list_colors)
plt.title('Mid', size=15)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(lateH, pos=late_node_positions,  node_size=20,  edge_color = "lightgrey", node_color = late_list_colors)
plt.title('Late', size=15)
plt.show()

In [ ]:
nog=community.induced_graph(noPART,noH)
create_corr_network_1(G=nog)

In [ ]:
ovg=community.induced_graph(ovPART,ovH)
create_corr_network_1(G=ovg)

In [ ]:
obg=community.induced_graph(obPART,obH)
create_corr_network_1(G=obg)

In [ ]:
earlyg=community.induced_graph(earlyPART,earlyH)
create_corr_network_1(G=earlyg)

In [ ]:
midg=community.induced_graph(midPART,midH)
create_corr_network_1(G=midg)

In [ ]:
lateg=community.induced_graph(latePART,lateH)
create_corr_network_1(G=lateg)

### Summmary of results so far... 
Looks like the obese have one less module compared to the overweight and the normal weight 
Need to assess what each node is comprised of  


# Visualizations

In [ ]:
# Define node positions data structure (dict) for plotting
no_node_positions = {node[0]: (node[1]['X'], -node[1]['Y'], node[1]['Z']) for node in noH.nodes(data=True)}
no_node_colors = {node[0]: (node[1]['color']) for node in noH.nodes(data=True)}

# Preview of node_positions with a bit of hack (there is no head/slice method for dictionaries).
#dict(list(node_colors.items())[0:5])
#print(G.node(data=True))
list_colors=list(node_colors.values())

In [ ]:
# Download data
# arch = np.load(download_file('phase_sync_delta.npz', astype='example_data'))
nodes, edges = arch['nodes'], arch['edges']
# Create the scene with a black background
sc = SceneObj(size=(1500, 600))

In [ ]:
print(arch['nodes'].shape)
print(arch['edges'].shape)

In [ ]:
# Download data
# arch = np.load(download_file('phase_sync_delta.npz', astype='example_data'))
nodes, edges = np.asarray(list(no_node_positions.values())), mean_no
# Create the scene with a black background
sc = SceneObj(size=(1500, 600))

In [ ]:
# Coloring method
color_by = 'strength'
# Because we don't want to plot every connections, we only keep connections
# above .7
select = edges > .5
# Define the connectivity object
c_default = ConnectObj('default', nodes, edges, select=select, line_width=2.,
                       cmap='Spectral_r', color_by=color_by)
# Then, we define the sources
s_obj = SourceObj('sources', nodes, color='#ab4642', radius_min=15.)
sc.add_to_subplot(c_default, title='Color by connectivity strength')
# And add connect, source and brain objects to the scene
sc.add_to_subplot(s_obj)
sc.add_to_subplot(BrainObj('B3'), use_this_cam=True)

In [ ]:
# Coloring method
color_by = 'count'
# Weak connections -> alpha = .1 // strong connections -> alpha = 1.
dynamic = (.1, 1.)
# Define the connectivity and source object
c_count = ConnectObj('default', nodes, edges, select=select, line_width=4.,
                     color_by=color_by, antialias=True, dynamic=dynamic)
s_obj_c = SourceObj('sources', nodes, color='olive', radius_min=10.,
                    symbol='square')
# And add connect, source and brain objects to the scene
sc.add_to_subplot(c_count, row=0, col=1,
                  title='Color by number of connections per node')
sc.add_to_subplot(s_obj_c, use_this_cam=True, row=0, col=1)
sc.add_to_subplot(BrainObj('B3'), use_this_cam=True, row=0, col=1)

In [ ]:
# First, we take a copy of the connectivity array
edges_copy = edges.copy()
# Then, we force edges to take fixed values
# ====================   =========  ===========
# Condition              New value  Color
# ====================   =========  ===========
# edges >= 0.8              4.      red
# edges in [.78, .8[        3.      orange
# edges in [.74, .78[       2.      blue
# Others                    -       lightgray
# ====================   =========  ===========
edges_copy[edges_copy >= .8] = 4.
edges_copy[np.logical_and(edges_copy >= .78, edges_copy < .8)] = 3.
edges_copy[np.logical_and(edges_copy >= .74, edges_copy < .78)] = 2.
# Now we use a dctionary to set one color per value.
ccol = {
    None: 'lightgray',
    2.: 'blue',
    3.: 'orange',
    4.: 'red'
}

# Define the connectivity and source objects
c_cuscol = ConnectObj('default', nodes, edges_copy, select=edges > .7,
                      custom_colors=ccol)
s_obj_cu = SourceObj('sources', nodes, color='slategray', radius_min=10.,
                     symbol='ring')
# Add objects to the scene
sc.add_to_subplot(c_cuscol, row=0, col=2, title='Custom colors')
sc.add_to_subplot(s_obj_cu, row=0, col=2)
sc.add_to_subplot(BrainObj('white'), use_this_cam=True, row=0, col=2)

# Finally, display the scene
sc.preview()
sc.screenshot('/Users/gracer/Google Drive/ABCD/tmp/figlayout.png', dpi=600)

## Saving the data

In [ ]:
no_ROIs = list(noPART.keys())
no_MODS = list(noPART.values())

ov_ROIs = list(ovPART.keys())
ov_MODS = list(ovPART.values())

ob_ROIs = list(obPART.keys())
ob_MODS = list(obPART.values())


In [ ]:
nodf = pd.DataFrame(no_MODS)
nodf['ROI'] = no_ROIs
# nodf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/normal_Module.csv", sep=',', index=False)

In [ ]:
ovdf = pd.DataFrame(ov_MODS)
ovdf['ROI'] = ov_ROIs
# ovdf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/overweight_Module.csv", sep=',', index=False)

In [ ]:
obdf = pd.DataFrame(ob_MODS)
obdf['ROI'] = ob_ROIs
# obdf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/obese_Module.csv", sep=',', index=False)

In [ ]:
early_ROIs = list(earlyPART.keys())
early_MODS = list(earlyPART.values())

mid_ROIs = list(midPART.keys())
mid_MODS = list(midPART.values())

late_ROIs = list(latePART.keys())
late_MODS = list(latePART.values())


In [ ]:
earlydf = pd.DataFrame(early_MODS)
earlydf['ROI'] = early_ROIs
earlydf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/early_Module.csv", sep=',', index=False)

In [ ]:
middf = pd.DataFrame(mid_MODS)
middf['ROI'] = mid_ROIs
middf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/mid_Module.csv", sep=',', index=False)

In [ ]:
latedf = pd.DataFrame(late_MODS)
latedf['ROI'] = late_ROIs
latedf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/late_Module.csv", sep=',', index=False)

## Parcelation 
Through BIAC https://wiki.biac.duke.edu/biac:analysis:resting_pipeline

## Individual and Group Matrices
Network-level analysis will be performed with inividual correltion matrices

## Thresholding
In accordance with van den Heuvel et al. 2017, we will examine and test statistical differences in functional connectivity (FC) defined as the mean of the correlation matrix. FC will be included in statistical tests between groups.

## Partitioning
Will partition full 264 connectome into modules using louvain algorithm. 

## Check the partition
Will use normalized mutual information to assess similarity between network assignments. NMI measures information shared between two probability distribution functions, specifically measuring how much knowing one distribution leads to certainty ofthe other. Permuted the labels of individual matrices between contrasts 1,000 times to generate a null distribution of NMI values for each contrast. Matrices between groups were randomly shuffled and partitioned into functional networks, and NMI was calculated.   
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html#sklearn.metrics.normalized_mutual_info_score

## Connectivity Strength
Caluclate Euclidean distance for each ROI-ROI pair. Linear regression with distance as a predictor of connectivity strength between groups.  
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.euclidean_distances.html

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
X = [[0, 1], [1, 1]]
print(X)
# distance between rows of X
euclidean_distances(X, X)

# get distance to origin
# euclidean_distances(X, [[0, 0]])



### Within network changes
All within network pairwise relationships were averaged per group. Two-tailed T-test to assess differences. Bonferroni corrections as needed.

### Between network changes
Average connectivity is calculated per network. Compare the between network interactions. 

## Pickling data to save it

In [ ]:
# sfile = bz2.BZ2File('/Users/gracer/Google Drive/ABCD/tmp/smallerfile', 'w')
# pickle.dump(GRAPHS, sfile)
GRAPHS['BIGdf'] = BIGdf
GRAPHS['maxes'] = maxes
pickle.dump(saveme, open('/Users/gracer/Google Drive/ABCD/tmp/just_mean', 'wb'), protocol=4)